# ПРОЕКТ 1 "Исследование надёжности заёмщиков"

Исходные данные:
По заданию кредитного отдела банка на основе статистики платежеспособности клиентов необходимо определить, влияет ли семейное положение и количество детей на возвращаемость кредита в срок.

Результаты исследования будут учтены при построении модели кредитного скоринга.

## Состав проекта

1.   [Шаг 1. Изучение исходных данных](#prestudy)
2.   [Шаг 2. Предобработка данных](#prep)<br>
  2.1. Обработка пропусков<br>
  2.2. Замена типа данных<br>
  2.3. Обработка дуликатов<br>
  2.4. Лемматизация<br>
  2.5. Исправление ошибок в данных<br>
  2.6. Категоризация данных<br>

3.   [Шаг 3. Анализ данных](#analysis)<br>
  3.1: Есть ли зависимость между наличием детей и возвратом кредита в срок?<br>
  3.2: Есть ли зависимость между семейным положением и возвратом кредита в срок?<br>
  3.3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?<br>
  3.4: Как разные цели кредита влияют на его возврат в срок?<br>

4.   [Шаг 4. Общий вывод](#resume)

## <a class='anchor' id='prestudy'>Шаг 1. Изучение исходных данных</a>

In [ ]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
TOTAL_LINES = data['children'].count()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод по шагу 1

Первый вывод исходя из типа данных в таблице:
- days_employed трудовой стаж в днях имеет формат float64, а число д.б. целым! Проверить!
- days_employed трудовой стаж days_employed - заполнен не для всех 21525 записей (есть пропуски, разобраться!)
- debt по логике в столбце (долги по кредиту) д.б. тип bool вместо int64
- total_income ежемес. доход указан не для всех записей - есть пропуски!
- ОШИБКА в children "-1" - исправить

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хорошее начало, так же, для большего понимания данных можно было бы использовать метод `describe`


## <a id='prep'>Шаг 2. Предобработка данных</a>

### 2.1. Обработка пропусков

In [ ]:
### Обработка пропусков

data['days_employed'].isnull().sum()
# Всего пропуской в days_employed 2174 строк

# Посмотрим на строки с пропусками, чтобы попытаться определить закономерности "на глаз"
data_employed_nan = data[data['days_employed'].isnull() == True]
data_employed_nan.head(15)
#data_employed_nan.describe()

# Проверим, совпадают ли строки с пропусками в колонке days_employed со строками в колонке total_income
data_total_income_nan = data[data['total_income'].isnull() == True]

# Соединим таблицы
#data.head()
#try:
#    data_nan_merged = data_employed_nan.merge(data_total_income_nan, on='children', how='left')
#    print('Таблицы имеют одинаковые строки')
#except:
#    print('Таблицы имеют разные строки!!!')
# ---> Вывод сравнения: пропуски в обеих колонках - это одни и те же строки! И это хорошо!


# Для дальнейшего заполнения пропусков, сохраним средние значения колонок days_employed и total_income
days_employed_nonan = data[data['days_employed'].isnull() == False]
days_employed_nonan_mean = int(days_employed_nonan['days_employed'].mean()) # = 63046 (АХТУНГ, это более 172 ЛЕТ!!!)
total_income_nonan_mean = int(days_employed_nonan['total_income'].mean()) # = 167422


# Чтобы понять возможные причины появления пропусков в данных, попробуем поманипулировать таблицей data_employed_nan
#data_employed_nan['education'].value_counts()
# -> Больше всего пропуской у СОТРУДНИКОВ и КОМПАНЬОНОВ, т.е. предположение, что доход не указан 
# -> в основном у пенсионеров и студентов (т.е. неработающих) - несостоятелен
# ---> После проверки по основным колонкам выявить какие-то закономерности "на глаз" не удалось.
# ---> Следовательно, делаем вывод, что причина появления пропусков - случайная ошибка в данных.

# Для начала заменим NaN на пустое значение ''
data['days_employed'] = data['days_employed'].fillna(value='')

# Сделаем то же самое для колонки total_income
data['total_income'] = data['total_income'].fillna(value='')


# Теперь заменим наши бывшие пропуски на средние значения в каждой колонке
data['days_employed'] = data['days_employed'].replace('', days_employed_nonan_mean)
data['total_income'] = data['total_income'].replace('', total_income_nonan_mean)

data.head(15)

# Проверим типы данных в колонках
data.info()
# -> Тип данных с колонках с пропусками изменился на object. Нужно вернуть int !!!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Вывод 2.1

1. определите и заполните пропущенные значения:
- опишите, какие пропущенные значения вы обнаружили;
- приведите возможные причины появления пропусков в данных;
- объясните, по какому принципу заполнены пропуски;

Пропущенные значения выявлены в колонках 'days_employed' и 'total_income'.

Всего таких строк 2174, т.е. 10,1% от общего числа 21525 строк. Это много.
Возможные причины появления пропусков в колонке days_employed -
    от возраста не зависит (распределение равномерное) + 101 строка с dob_years=0 - ОШИБКА!
    возможно зависит от образования (со средн.обр=1408+67+65=1540 eq 70,8% от пропусков, с высшим=23+25+496=544 eq 25% от пропусков)
    от пола (пропуски M=690, F=1484; всего M=7288, F=14236, одна строка gender='XNA' id 10701) - ОШИБКА!
    ...
Заполнение пропусков:
    для колонки трудовой стаж (дн) days_employed - на среднее по колонке
    для колонки ежемес.доход total_income - на среднее по колонке

### 2.2. Замена типа данных

In [ ]:
# Используем метод .astype() для изменения типа данных
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### Вывод 2.2

Поскольку колонка days_employed содержит информацию о днях трудового стажа, то логично, что в ней должны содержаться ЦЕЛЫЕ числа.
До замены тип данных, содержащихся в колонке days_employed был float (нецелое число с плавающей точкой).
Поэтому заменил значения в столбце days_employed на int (целое число).

ДОПОЛНИТЕЛЬНО
поменяем тип данных в колонке total_income (доход в месяц) с float на int, поскольку информация "до копеек" тут не требуется.

Во всех остальных колонках тип данных корректный и замены не требует!

### 2.3. Обработка дубликатов

In [ ]:
 # 4. Исправим регистры в education
data['education'] = data['education'].str.lower()
#data['education'].value_counts()
# среднее=15233, высшее=5260, неоконченное высшее=744, начальное=282, уч.степень=6, ИТОГО-21525

# За одно 'для красоты' понизим регистр для столбца family_status
data['family_status'] = data['family_status'].str.lower()

# Посмотрим на строки, которые дублируются
data_dupl = data[data.duplicated(keep=False)]
# Посчитаем, сколько всего дублирующихся строк
data_dupl.count() # -> Всего 137 строк, из них 71 - повторы (keep='last')

# применим drop_deplicates() совместно с reset_index() для удаления дубликатов
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated(keep=False).sum()
#data.head(15)

0

#### Вывод 2.3

Всего дубликатов методом .duplicated() выявлено 137 штуки (после правки регистра в education), из них повторов 71.

При просмотре выявлено, что это реально дублирующиеся строки, поэтому решено их удалить. 
Удаляю методом drop_duplicates().

Вероятная причина появления дубликатов - объединение нескольких баз, либо нескольких версий одной базы.

### 2.4. Лемматизация

In [ ]:
from pymystem3 import Mystem
m = Mystem()
#data['lemmas'] = ' '.join(m.lemmatize(str(data['purpose'])))
# получаю список уникальных значений столбца purpose (причина для кредита)
purpose_unique = data['purpose'].unique()

# проверяю полученный список
#print(purpose_unique)

# создаю в цикле новый список уникальных лемм
lemmas = []
for i in purpose_unique:
#    result = m.lemmatize(i)
    result = ''.join(m.lemmatize(i)).strip()
    # Соединяем леммы из дург с другой (вместо получения каждого слова как отдельного значения) [ ''.join() ],
    # а также удаляем все пробелы оператором .strip() 
    lemmas.append(result)
#print(lemmas)
# Получили Series с лемматизированными уникальными значениями всех строк таблицы data


#### Вывод 2.4

Типовых запросов не так уж много: жильё/недвижимость, автомобиль, свадьба, образование и другое

### 2.5. Исправление ошибок в данных

In [ ]:
                                    ##### ИСПРАВИМ ОШИБКИ В ДАННЫХ #####

                                    # 1. Поправим значение в количестве детей
data['children'] = data['children'].replace(-1, 1) # предполагаю, что это ошибка ввода, т.к. единичный случай
#data['children'].unique()

                                    # 6. Исправим значение в gender 
data['gender'] = data['gender'].replace('XNA', 'F') # меняю на 'F' т.к. по статистике их в два раза больше
#data['gender'].value_counts()

                                    # 2. Поправим значения в трудовом стаже days_employed
## Предполагаю, что отрицательные значения - ошибка
# Проверим, сколько всего строк с отрицательным days_employed
#data[data['days_employed'] < 0]['days_employed'].count()
# Результат: всего 15906 строк с отрицательным days_employed (т.е. почти 74%) Это ОЧЕНЬ Много!!!
# очевидно присутствует какая-то системная ошибка

# Проверим, как распределяется по возрастам
#data[data['days_employed'] < 0]['dob_years'].value_counts()
# Результат: Распределение более-менее равномерное

# Проверим, как распределяется по типу образования
#data[data['days_employed'] < 0]['education'].value_counts()
# Результат: среднее=10899, высшее=419, неоконч.высшее=640, начальное=168, уч.степень=4
# Вывод: Строгой зависимости нет

# Проверим, как распределяется по профессии income_type
#data[data['days_employed'] < 0]['income_type'].value_counts()
# Результат: сотрудник=10014, компаньон=4577, госслужащий=1312, предприниматель=1, в.декрете=1, студент=1
# Вывод: Строгой зависимости нет

# Проверим диапазон отрицательных значений
#data[data['days_employed'] < 0].sort_values(by='days_employed').head()
#data[data['days_employed'] < 0].sort_values(by='days_employed').tail(50)
# Результат: Отрицательные значения в диапазоне от -18388 до -24

#data_minus = data[data['days_employed'] < -80]
#data_minus[data_minus['days_employed'] > -90].sort_values(by='days_employed')

# Проверим, если ли связь отрицательных значений стажа days_employed с неотдачей кредита
#data[data['days_employed'] < 0]['debt'].value_counts()
# Результат: 14517 - не имело задолженности, 1389 - имело. Вывод: Связи нет!
#data['days_employed'].min()
### ОБЩИЙ ВЫВОД по столбцу days_employed: Значения неадекватные. В расчёт не принимать!!!

                                     # 3. Исправим значения в dob_years (возраст заёмщика)
# Найтём средний возраст для тех, у кого он указан, запишем его в переменную dob_years_mean
data_age_nozero = data[data['dob_years'] != 0]
dob_years_mean = data_age_nozero['dob_years'].mean()       # результат 43.4974
#dob_years_mean = int(data_age_nozero['dob_years'].mean()) # результат 43
#print(dob_years_mean)
# Заменим возраст 0 на средний возраст в таблице
data['dob_years'] = data['dob_years'].replace(0, dob_years_mean)
#data['dob_years'].value_counts()
#data[data['dob_years'] == 0]['family_status'].value_counts()

                                    ##### ОШИБКИ В ДАННЫХ ИСПРАВЛЕНЫ #####

NameError: ignored

### 2.6. Категоризация данных

In [ ]:
# Выделим 4 категории в зависимости от цели получения кредита (и создадим одну для проверки)

# Альтернативный вариант функции БЕЗ обращения к новому (несозданному) столбцу 
def purpose_type(row):
    """
    Определяет тип причины для кредита исходя из условий:
    ключевое слово "жилье" или "недвижимость", то тип "недвижимость"
    ключ.сл. "автомобиль", то тип "авто"
    "образование", то "образование"
    "свадьба", то "свадьба"
    иначе, "другое"
    """
    lemmatized_purpose = m.lemmatize(row['purpose'])
    if 'жилье' in lemmatized_purpose or 'недвижимость' in lemmatized_purpose:
        return 'недвижимость'
    if 'автомобиль' in lemmatized_purpose:
        return 'авто'
    if 'образование' in lemmatized_purpose:
        return 'образование'
    if 'свадьба' in lemmatized_purpose:
        return 'свадьба'
    else:
        return 'прочее' # создаю на всякий случай как проверку

    
#purpose_type('приобретение автомоб')    
data['purpose_type'] = data.apply(purpose_type, axis='columns') #alt 'axis=1'
# Добавление аргументы AXIS='columns' заставляет метод .apple() работать не по колонкам (по умолчанию), 
# а пройтись по всем строкам

# Проверка
#data.tail(20)
#data['purpose_type'].unique()

# Посчитаем, сколько запросов в каждой категории
#data['purpose_type'].value_counts()
# В процентах
#pers = data[data['purpose_type'] == 'недвижимость']['purpose_type'].count()/TOTAL_LINES
#print(pers)
# Результат недвижимость=10840 (eq. 50,4%), авто=4315 (eq. 20%), образование=4022 (eq. 18,7%), 
#           свадьба=2348 (eq. 10,9%), ВСЕГО=21525


#data_pivot = data.pivot_table(index=['purpose_type', 'dob_years'], columns='gender',\
#                              values='education', aggfunc='count')
#data_pivot.head(35)

# Проверим зависимость возврата от пола
#pivot_gender_debt = data.pivot_table(index=['debt'], columns='gender',values='income_type', aggfunc='count')
#pivot_gender_debt.head()
# Процент невозврата  F - 7%  M - 10%

# Проверим зависимость возврата от профессии
#pivot_income_type_debt = data.pivot_table(index=['debt'], columns='income_type',values='gender', aggfunc='count')
#pivot_income_type_debt.head()
# Процент невозврата  безработный - 50%, в декрете - 100%, госслужащий - 6%, компаньон - 7%, пенсионер - 5,6%,
# предприниматель - 0%, сотрудник - 9,5%, студент - 0%
# при этом студентам, предпринимателям, безработным и в декрете - кредиты почти не выдают!!!

### Вывод по шагу 2

В итоге получили 4 категории причин получения кредита: 
- недвижимость (50,4% или 10840 запросов)
- авто (20,0% или 4315 запросов)
- образование (18,7% или 4022 запросов)
- свадьба (10,9% или 2348 запросов)


## <a class='anchor' id='analysis'>Шаг 3. Анализ данных</a>

### Вопрос 3.1: Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# Проверим, если ли зависимость, для этого создадим сводную таблицу
pivot_children_debt = data.pivot_table(index=['debt'], columns='children',values='gender', aggfunc='count')
pivot_children_debt.head(35)
# Результат:
    # без детей: нет долгов = 13086 (60% от 21525) всего бездетных 14149(92% от бездетных)
    #            есть долги = 1063 (5% от 21525) (8% от бездетных)
    # C детьми: нет долгов = 4420+1861+303+37+9+68=6698 (31% от 21525) всего с-детьми 7376 (91% от с-детьми)
    #            есть долги = 445+194+27+4+0+8=678 (3% от 21525) (9% от с-детьми)

children,0,1,2,3,4,5,20
debt,,,,,,,
0,13028.0,4410.0,1858.0,303.0,37.0,9.0,68.0
1,1063.0,445.0,194.0,27.0,4.0,NaN,8.0


#### Ответ 3.1

Корреляции между наличием детей и возвратом кредита нет (процент возврата примерно одинаковый).

Результат:
    # без детей: нет долгов = 13086 (60% от 21525) всего бездетных 14149(92% от бездетных)
    #            есть долги = 1063 (5% от 21525) (8% от бездетных)
    # C детьми: нет долгов = 4420+1861+303+37+9+68=6698 (31% от 21525) всего с-детьми 7376 (91% от с-детьми)
    #            есть долги = 445+194+27+4+0+8=678 (3% от 21525) (9% от с-детьми)

### Вопрос 3.2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# Проверим, если ли зависимость, для этого создадим сводную таблицу
pivot_famstat_debt = data.pivot_table(index=['debt'], columns='family_status',values='gender', aggfunc='count')
pivot_famstat_debt.head(35)
# Результат:
    # не женат: нет долгов = 2539; всего холостых 2813(90% от холостых)
    #           есть долги = 274; (10% от холостых)
    # в разводе: нет долгов = 1110; всего в-разводе 1195 (93% от в-разводе)
    #            есть долги = 85; (7% от в-разводе)
    # вдова:     нет долгов = 897; всего вдова 960 (93% от вдова)
    #            есть долги = 63; (7% от вдова)
    # гр.брак:   нет долгов = 3789; всего гр.брак 4177 (91% от гр.брак)
    #            есть долги = 388; (9% от гр.брак)
    # женат:     нет долгов = 11449; всего женат 12380 (92% от женат)
    #            есть долги = 931; (8% от женат)

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
debt,,,,,
0,1110,896,3763,11408,2536
1,85,63,388,931,274


#### Ответ 3.2

Слабая корреляции между семейным положением и возвратом кредита отслеживается.

Результат:
    # не женат: нет долгов = 2539; всего холостых 2813(90% от холостых)
    #           есть долги = 274; (10% от холостых)
    # в разводе: нет долгов = 1110; всего в-разводе 1195 (93% от в-разводе)
    #            есть долги = 85; (7% от в-разводе)
    # вдова:     нет долгов = 897; всего вдова 960 (93% от вдова)
    #            есть долги = 63; (7% от вдова)
    # гр.брак:   нет долгов = 3789; всего гр.брак 4177 (91% от гр.брак)
    #            есть долги = 388; (9% от гр.брак)
    # женат:     нет долгов = 11449; всего женат 12380 (92% от женат)
    #            есть долги = 931; (8% от женат)
    
То есть самые надежные заемщики - в разводе, вдовы и женатые
Самые ненадежные заемщики - холостые и в гражданском браке

### Вопрос 3.3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# Сначала разделим на статусы в зависимости от заработка
#data.sort_values(by='total_income').tail()
# минимальное значение 20'667
# максимальное значение 2'265'604
def income_status(row):
    '''
    Определяет статус исходя из уровня заработка
    До 50т.р. - минимум
    менее 100т.р. - ниже среднего
    менее 300т.р. - средний
    менее 1млн.р. - выше среднего
    более 1млн.р. - максимум
    '''
    total_income = row['total_income']
    if total_income < 50000: # доход менее 50тр - минимум
        return 'минимум'
    if total_income < 100000: # доход от 50тр до 100тр - ниже среднего
        return 'ниже среднего'
    if total_income < 200000: # доход от 100тр до 200тр - средний
        return 'средний'
    if total_income < 500000: # доход от 200тр до 500тр - выше среднего
        return 'выше среднего'
    else:
        return 'максимум' # доход более 500тр в месяц - максимум
    
# более детальная разбивка даёт практически такой же результат (результат 2)
#    total_income = row['total_income']
#    if total_income < 30000: # доход менее 30тр - минимум
#        return 'минимум'
#    if total_income < 50000: # доход от 30тр до 50тр - низкий
#        return 'низкий'
#    if total_income < 100000: # доход от 50тр до 100тр - ниже среднего
#        return 'ниже среднего'
#    if total_income < 150000: # доход от 100тр до 150тр - средний
#        return 'средний'
#    if total_income < 300000: # доход от 150тр до 300тр - выше среднего
#        return 'выше среднего'
#    if total_income < 1000000: # доход от 300тр до 1млн - высокий
#        return 'высокий'
#    else:
#        return 'максимум' # доход более 1млн в месяц - максимум
    

data['total_income_status'] = data.apply(income_status, axis='columns')
#data.head()

# Создадим сводную таблицу для проверки зависимости
pivot_income_debt = data.pivot_table(index=['debt'], columns='total_income_status',values='gender', aggfunc='count')
pivot_income_debt.head()
# Результат 1:
    # минимум: нет долгов = 349; всего минимум 372(94% от минимум)
    #          есть долги = 23; (6% от минимум)
    # ниже_ср: нет долгов = 3760; всего ниже_ср 4091 (92% от ниже_ср)
    #          есть долги = 331; (8% от ниже_ср)
    # средний: нет долгов = 10967; всего средний 11996 (91% от средний)
    #          есть долги = 1029; (9% от средний)
    # выше_ср: нет долгов = 4500; всего выше_ср 4844 (93% от выше_ср)
    #          есть долги = 344; (7% от выше_ср)
    # максимум:нет долгов = 208; всего максимум 222 (94% от максимум)
    #          есть долги = 14; (6% от максимум)

# Результат 2 - невозврат, %:
    # минимум 9% (20-2)
    # низкий 6% (329-21)
    # ниже среднего 8% (3760-331)
    # средний 9% (5213-491)
    # выше среднего 8% (9086-790)
    # высокий 7% (1353-104)
    # максимум 8% (23-2)

total_income_status,выше среднего,максимум,минимум,ниже среднего,средний
debt,,,,,
0,4500,208,349,3760,10896
1,344,14,23,331,1029


#### Ответ 3.3

Очень небольшая зависимость возврата кредита от уровня дохода есть:
Самый высокий процент невозврата:
9% в группе с доходами от 100тр до 200тр
8% в группе с доходами от 50тр дл 100тр
Самый низкий процент невозврата в группе с доходами менее 50тр - всего 6%.

### Вопрос 3.4: Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# Создадим сводную таблицу для проверки зависимости
pivot_purpose_debt = data.pivot_table(index=['debt'], columns='purpose_type',values='gender', aggfunc='count')
pivot_purpose_debt.head()

purpose_type,авто,недвижимость,образование,свадьба
debt,,,,
0,3903,10029,3643,2138
1,403,782,370,186


#### Ответ 3.4

Результаты:
Невозврат по группам, %
    Авто 9% (3912-403)
    Недвижимость 7% (10058-782)
    Образование 9%(3652-370)
    Свадьба 8% (2162-186)
    
Вывод: Самая надежная цель для получения кредита - категория "недвижимость"
Самая ненадежная цель - "авто" и (как ни странно) "образование"

## <a class='anchor' id='resume'>Шаг 4. Общий вывод</a>

Возврат выданного кредита зависит:
    - от семейного положение (самые надежные заемщики - это "в разводе", "вдовы/вдовцы" и "женатые/замужем", а самые ненадежные заемщики - "холостые" и "в гражданском браке"
    - от цели кредита (самые надежные заемщики берут кредит на покупку или ремонт недвижимости, а самые ненадежные - на автомобиль и образование.
Возврат выданного кредита не зависит:
    - от наличия детей (возвращают одинаково и заемщики с детьми, и бездетные)
    - от уровня дохода (заемщики с разным уровнем дохода имеют примерно одинаковый процент возврата кредита в срок)
    
Таким образом, портрет идеального заемщика: Женщина-пенсионер или госслужащая, состоящая в браке, которая планирует приобретение недвижимости.
Потртет нежелательного заемщика: Холостой мужчина, сотрудник по найму, которых хочет взять кредит на автомобиль.